In [86]:
#Para obtener el csv de las estaciones de carga de EE.UU. con la información que necesitamos
import pandas as pd
import numpy as np

df = pd.read_csv('eeuu_fuel_stations.csv', low_memory=False)
df = df.sample(frac=1, random_state=np.random.RandomState())

df2 = df[[ "Station Name", "Latitude", "Longitude", "EV Level2 EVSE Num"]]
#Eliminamos los NULL
df2 = df2[df["EV Level2 EVSE Num"] > 0]
df2.to_csv('eeuu_fuel_stations_processed.csv', index=False)
df2["Station Name"].astype(str)
df2


,Station Name,Latitude,Longitude,EV Level2 EVSE Num
10865,GWV STATION 03,41.257688,-75.807628,2.0
33172,RTS IRONDEQUOIT #2,43.209057,-77.601380,2.0
25922,UNICOPROPERTIES HARLEQUIN 1,39.607415,-104.901598,2.0
52377,YKF STATION 1,43.455215,-80.387007,2.0
38475,111 DUNSMUIR 111 DUNSMUIR,49.280552,-123.111687,2.0
...,...,...,...,...
29373,CU ANDERSON 19,33.794458,-117.850306,1.0
33340,CAPCOMFCU CHRGPOINT CT4 1,42.719104,-73.795364,2.0
5305,Sumner School District - Parking Lot,47.203981,-122.237313,3.0
1704,MAPLE LAWN 6 STATION 01,39.150946,-76.902700,2.0


In [87]:
df2["index"] = df2.index + 1
df2.head()

,Station Name,Latitude,Longitude,EV Level2 EVSE Num,index
10865,GWV STATION 03,41.257688,-75.807628,2.0,10866
33172,RTS IRONDEQUOIT #2,43.209057,-77.601380,2.0,33173
25922,UNICOPROPERTIES HARLEQUIN 1,39.607415,-104.901598,2.0,25923
52377,YKF STATION 1,43.455215,-80.387007,2.0,52378
38475,111 DUNSMUIR 111 DUNSMUIR,49.280552,-123.111687,2.0,38476


In [88]:
#Para importar la información a DynamoDB

#Importacion de las electrolineras
import boto3
from decimal import Decimal
import json

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
electrolineras_tabla = dynamodb.Table('Electrolinera')

#Eliminamos las entradas anteriores
response = electrolineras_tabla.scan()
for i in response['Items']:
    electrolineras_tabla.delete_item(
        Key={
            'ID_Electrolinera': i['ID_Electrolinera']
        }
    )

def import_electrolinera(row):

    item = {
            'ID_Electrolinera': row["index"],
            'nombre': row['Station Name'],
            'latitud': row['Latitude'],
            'longitud': row['Longitude'],
            'estaciones': row['EV Level2 EVSE Num'],
            'observaciones': "",
            'estado': 1
        }

    electrolineras_tabla.put_item(
        Item = json.loads(json.dumps(item), parse_float=Decimal)
    )


df_mini = df2.head(100)
df_mini.apply(import_electrolinera, axis=1)

10865    None
33172    None
25922    None
52377    None
38475    None
         ... 
40992    None
22627    None
15496    None
3172     None
18651    None
Length: 100, dtype: object

In [89]:
#Ahora generamos un punto de carga por cada electrolinera importada
df_mini["puntos_de_carga"] = df_mini["EV Level2 EVSE Num"].apply(lambda x: [i for i in range(1, int(x)+1)])
df_mini = df_mini.explode("puntos_de_carga")
df_mini


/tmp/ipykernel_13746/1887988630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mini["puntos_de_carga"] = df_mini["EV Level2 EVSE Num"].apply(lambda x: [i for i in range(1, int(x)+1)])


,Station Name,Latitude,Longitude,EV Level2 EVSE Num,index,puntos_de_carga
10865,GWV STATION 03,41.257688,-75.807628,2.0,10866,1
10865,GWV STATION 03,41.257688,-75.807628,2.0,10866,2
33172,RTS IRONDEQUOIT #2,43.209057,-77.601380,2.0,33173,1
33172,RTS IRONDEQUOIT #2,43.209057,-77.601380,2.0,33173,2
25922,UNICOPROPERTIES HARLEQUIN 1,39.607415,-104.901598,2.0,25923,1
...,...,...,...,...,...,...
22627,Loretto Motherhouse Infirmary,37.664534,-85.400165,2.0,22628,2
15496,Municipalité de Sainte-Germaine-Boulé,48.603858,-79.115185,1.0,15497,1
3172,Roaring Fork High School,39.387753,-107.203871,2.0,3173,1
3172,Roaring Fork High School,39.387753,-107.203871,2.0,3173,2


In [90]:

#Importacion de los puntos de carga
puntos_de_carga_tabla = dynamodb.Table('PuntoCarga')

#Eliminamos las entradas anteriores
response = puntos_de_carga_tabla.scan()
for i in response['Items']:
    puntos_de_carga_tabla.delete_item(
        Key={
            'ID_PuntoCarga': i['ID_PuntoCarga']
        }
    )

def import_punto_de_carga(row):
        item = {
                'ID_PuntoCarga': row["index"]*1000 + row["puntos_de_carga"],
                'ID_Electrolinera': row["index"],
                'num_pc_electrolinera': row['puntos_de_carga'],
                'estado': 1,
                'uptime': 0
            }
    
        puntos_de_carga_tabla.put_item(
            Item = json.loads(json.dumps(item), parse_float=Decimal)
        )

df_mini.apply(import_punto_de_carga, axis=1)

10865    None
10865    None
33172    None
33172    None
25922    None
         ... 
22627    None
15496    None
3172     None
3172     None
18651    None
Length: 209, dtype: object